In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader




#Importing necessary libraries

In [68]:
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = os.listdir(root_dir)
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.images[idx])
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:

# Step 1: Data preprocessing
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Assuming you have your datasets in 'train_dataset' and 'test_dataset'
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Step 2: Define the CNN Architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Adjust channels as needed
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 512)  # Adjust size as needed
        self.fc2 = nn.Linear(512, 10)  # Adjust the number of classes as needed

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()

# Step 3: Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 4: Train the Model
for epoch in range(10):  # Number of epochs
    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize

print("Finished Training")

# Step 5: Evaluate the Model
# Similar loop as training but with test_loader and without backprop steps


In [69]:
# Define the path to the directory containing your images
root_dir = "../Training"

# Define transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize the image to 256x256 pixels
    transforms.ToTensor()            # Convert the image to a PyTorch tensor
])

# Create an instance of the ImageDataset class
dataset = ImageDataset(root_dir, transform=transform)

# Optionally, you can also use PyTorch's ImageFolder class which automatically loads images from subfolders
# dataset = ImageFolder(root=root_dir, transform=transform)

# Create a DataLoader to iterate over the dataset
batch_size = 32




In [70]:
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
